In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

df_epidural = pd.read_csv('EpiduralHemorrhageRemovedFlags.csv')
df_epidural = df_epidural.drop(['Unnamed: 0', 'Comments', 'Internal Notes', 'URL', 'Case ID', 'All Labels', 'Difficulty'], axis=1)
df_epidural.head(5)


In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0
    return image

image_directory = "C:\Users\wwwdz\OneDrive\Desktop\Spring Semester 2023\MATRIX METHODS\PROJECT\XN_Project\XN_Project\renders\intraventricular.zip"
image_paths = [os.path.join(image_directory, img_name) for img_name in df_epidural['Origin']]

X = np.array([preprocess_image(img_path) for img_path in image_paths])
y = pd.get_dummies(df_subdural['Labeling State']).values #???
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
classifier = SVC(kernel='linear', random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [7]:
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()


ValueError: could not convert string to float: 'ID_e732e6d17.jpg'

In [ ]:
# Load the dataset
df = pd.read_csv('IntraparenchymalHemorrhageRemovedFlags.csv')
df_intra = df.drop(['Unnamed: 0', 'Comments', 'Internal Notes', 'URL', 'Case ID', 'All Labels', 'Difficulty'], axis=1)
image_directory = "C:\Users\wwwdz\OneDrive\Desktop\Spring Semester 2023\MATRIX METHODS\PROJECT\XN_Project\XN_Project\\renders\intraparenchymal.zip"
image_paths = [os.path.join(image_directory, img_name) for img_name in df_intra['Origin']]

X = np.array([preprocess_image(img_path) for img_path in image_paths])
y = pd.get_dummies(df_intra['Labeling State']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Define the parameter grid for grid search
param_grid = {
    'Agreement': [1],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'Labeling State': ['Gold Standard']
}

# Create a SVM model and perform grid search
classifier = SVC()
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='f1')
grid_search.fit(X_train, y_train)

best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(X_test)

# Evaluate the performance of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))